In [ ]:
# !pip install google-genai faiss-cpu PyPDF2 python-dotenv numpy

In [ ]:
# Configuración y conexión con el LLM
# Propósito: Inicializar la API de Gemini, cargar la API key y seleccionar el modelo


import os
import logging
import numpy as np
import faiss
import PyPDF2
import google.genai as genai
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("No se encontró la API key en variables de entorno")

client = genai.Client(api_key=GOOGLE_API_KEY)

MODEL_NAME = "gemini-2.5-flash-lite"

print("Conexión con Gemini establecida.")


Conexión con Gemini establecida.


In [ ]:
# Carga del PDF y división en chunks
# Propósito: Leer la guía turística y dividirla en fragmentos para RAG

PDF_PATH = "TENERIFE.pdf"

def load_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

def chunk_text(text, chunk_size=500, overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

text = load_pdf(PDF_PATH)
chunks = chunk_text(text)

print(f"Texto cargado con {len(chunks)} chunks.")


Texto cargado con 38 chunks.


In [ ]:
# Embeddings y FAISS
# Propósito: Convertir los chunks en vectores y construir un índice FAISS para búsqueda rápida


def get_embedding(text):
    response = client.models.embed_content(
        model="gemini-embedding-001",
        contents=text
    )
    return np.array(response.embeddings[0].values, dtype="float32")

print("Generando embeddings...")

embeddings = [get_embedding(chunk) for chunk in chunks]

dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)

for emb in embeddings:
    index.add(emb.reshape(1, -1))

print(f"FAISS index creado con {index.ntotal} vectores.")


Generando embeddings...
FAISS index creado con 38 vectores.


In [ ]:
# Búsqueda semántica
# Propósito: Dada una pregunta, recuperar los chunks más relevantes del índice FAISS


def search_chunks(query, top_k=3):
    query_emb = get_embedding(query).reshape(1, -1)
    distances, indices = index.search(query_emb, top_k)
    
    results = []
    for i in indices[0]:
        results.append((i, chunks[i]))
    
    return results


In [ ]:
# Función externa simulada
# Propósito: Implementar la función get_weather para devolver predicciones de clima


logging.basicConfig(level=logging.INFO)

weather_function_schema = {
    "name": "get_weather",
    "description": "Obtiene la predicción del tiempo para una fecha concreta.",
    "parameters": {
        "type": "object",
        "properties": {
            "fecha": {
                "type": "string",
                "description": "Fecha en formato YYYY-MM-DD"
            }
        },
        "required": ["fecha"]
    }
}

def get_weather(fecha: str) -> str:
    logging.info(f"Llamada a get_weather con fecha: {fecha}")
    
    weather_data = {
        "2024-05-01": "Soleado, 26°C",
        "2024-05-02": "Parcialmente nublado, 24°C",
        "2024-05-03": "Lluvia ligera, 22°C",
    }
    
    try:
        return weather_data.get(fecha, "No hay datos disponibles para esa fecha.")
    except Exception as e:
        logging.error(f"Error en get_weather: {e}")
        return "Error consultando el servicio meteorológico."


In [ ]:
# Memoria de conversación
# Propósito: Guardar el historial y construir contexto para el LLM

chat_history = []
MAX_TURNS = 5

def build_conversation_context():
    history_text = ""
    for turn in chat_history[-MAX_TURNS:]:
        history_text += f"Usuario: {turn['user']}\n"
        history_text += f"Asistente: {turn['assistant']}\n"
    return history_text


In [ ]:
# Función principal de diálogo
# Propósito: Combinar RAG, multiturno y function call para generar respuestas


import re

def chat_rag(user_input, temperature=0.7, max_tokens=500):
    
    # 1️⃣ Detectar intención clima
    if "clima" in user_input.lower() or "tiempo" in user_input.lower():
        match = re.search(r"\d{4}-\d{2}-\d{2}", user_input)
        if match:
            fecha = match.group()
            weather_response = get_weather(fecha)
            chat_history.append({"user": user_input, "assistant": weather_response})
            return f"Predicción del tiempo para {fecha}: {weather_response}"
    
    # Recuperación RAG
    retrieved_chunks = search_chunks(user_input)
    
    context_text = ""
    for idx, chunk in retrieved_chunks:
        context_text += f"\n[Chunk {idx}]\n{chunk}\n"
    
    # Historial multiturno
    conversation_context = build_conversation_context()

    prompt = f"""
Eres un asistente turístico experto en Tenerife.

Historial:
{conversation_context}

Contexto relevante:
{context_text}

Pregunta:
{user_input}

Responde citando la fuente como:
Fuente: Chunk X
"""

    response = client.models.generate_content(
        model=MODEL_NAME,
        contents=prompt,
        config={
            "temperature": temperature,
            "max_output_tokens": max_tokens,
        }
    )

    text = (response.text or "").strip()

    chat_history.append({"user": user_input, "assistant": text})

    return text


In [ ]:
# Pruebas del asistente
# Propósito: Probar que RAG, multiturno y función clima funcionan correctamente

print(chat_rag("Háblame del Teide"))
print(chat_rag("¿Qué playas cercanas hay?"))
print(chat_rag("¿Qué tiempo hará el 2024-05-01?"))
print(chat_rag("¿Qué tiempo hará el 2024-12-01?"))


INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-embedding-001:batchEmbedContents "HTTP/1.1 200 OK"
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"


El Teide es una visita obligada si estás en Tenerife. La recomendación personal es subir por la carretera TF24 (carretera de La Esperanza) desde la rotonda de Padre Anchieta en La Laguna.

En el camino, puedes parar en el Mirador de La Tarta, llamado así por la forma en que la piedra de la curva parece una tarta con diferentes colores. Desde allí, si el norte está nublado, podrás disfrutar de un espectacular mar de nubes.

Para subir al Teide, lo ideal es llegar hasta el Parador de las Cañadas del Teide y aparcar allí para dirigirte al mirador de La Ruleta, desde donde se obtienen las vistas más famosas de la isla. Si deseas ascender hasta el pico del Teide, puedes hacerlo utilizando los teleféricos desde este punto.

Para obtener más información sobre el Teide, puedes visitar el Centro de Visitantes de El Portillo, que es gratuito. También existe la opción de subir de noche.

Fuente: Chunk 19
Fuente: Chunk 20
Fuente: Chunk 21


INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-embedding-001:batchEmbedContents "HTTP/1.1 200 OK"
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
INFO:root:Llamada a get_weather con fecha: 2024-05-01
INFO:root:Llamada a get_weather con fecha: 2024-12-01


En la zona de ocio, te recomendaría la Playa de Torviscas o la Playa de Troya. Encima de la Playa de Troya se encuentra el Monkey Beach Club, uno de los beach clubs más populares de Tenerife.
Fuente: Chunk 23

De la zona de Los Cristianos, te recomendaría la Playa de Los Cristianos o la Playa de Las Vistas.
Fuente: Chunk 30
Predicción del tiempo para 2024-05-01: Soleado, 26°C
Predicción del tiempo para 2024-12-01: No hay datos disponibles para esa fecha.


In [10]:
print("Número de chunks:", len(chunks))
print("Número de vectores en FAISS:", index.ntotal)
print("Historial actual:", len(chat_history))


Número de chunks: 38
Número de vectores en FAISS: 38
Historial actual: 4
